In [4]:
# Importiert die notwendingen Pakete
import tensorflow as tf
import tensorflow.keras as keras

## Einfaches GAN auf Basis eines Neural Networks

In [42]:
generator = keras.models.Sequential([
                                     keras.layers.Dense(100, activation="selu", input_shape=[30]),
                                     keras.layers.Dense(150, activation="selu"),
                                     keras.layers.Dense(28 * 28, activation="sigmoid"),
                                     keras.layers.Reshape([28,28])
                                    ], name="Generator")

discriminator = keras.models.Sequential([
                                         keras.layers.Flatten(input_shape=[28,28]),
                                         keras.layers.Dense(150, activation="selu"),
                                         keras.layers.Dense(100, activation="selu"),
                                         keras.layers.Dense(1, activation="sigmoid")
                                        ], name="Discriminator")

gan = keras.models.Sequential([generator, discriminator], name="GAN")

In [22]:
discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
discriminator.trainable = False

gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

In [25]:
def train_gan(gan, dataset, batch_size, codings_size, number_of_epochs):
    generator, discriminator = gan.layers
    
    for epoch in range(number_of_epochs):
        
        for X_batch in dataset:
            # Phase 1 - Den Discriminator trainieren
            noise = tf.random.normal(shape=[batch_size, codings_size])
            generated_images = generator(noise)
            X_fake_and_real = tf.concat([generated_images, X_batch])
            y1 = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)
            discriminator.trainable = True
            discriminator.train_on_batch(X_fake_and_real, y1)
            
            # Phase 2 - Den Generator trainieren
            noise = tf.random.normal(shape=[batch_size, codings_size])
            y1 = tf.constant([[1.]] * batch_size)
            discriminator.trainable = False
            gan.train_on_batch(noise, y2)

In [26]:
train_gan(gan, dataset, batch_size, codings_size, number_of_epochs)

SyntaxError: invalid syntax (1231087050.py, line 1)

## Einfaches GAN auf Basis eines Convolutional Neural Networks

In [40]:
codings_size = 100

generator = keras.models.Sequential([
                                     keras.layers.Dense(7 * 7 * 128, input_shape=[codings_size]),
                                     keras.layers.Reshape([7,7,128]),
                                     keras.layers.BatchNormalization(),
                                     keras.layers.Conv2DTranspose(64, kernel_size=5, strides=2, padding="same", activation="selu"),
                                     keras.layers.BatchNormalization(),
                                     keras.layers.Conv2DTranspose(1, kernel_size=5, strides=2, padding="same", activation="tanh")
                                    ], name="Generator")

discriminator = keras.models.Sequential([
                                         keras.layers.Conv2D(64, kernel_size=5, strides=2, padding="same", activation=keras.layers.LeakyReLU(0.2), input_shape=[28,28,1]),
                                         keras.layers.Dropout(0.4),
                                         keras.layers.Conv2D(128, kernel_size=5, strides=2, padding="same", activation=keras.layers.LeakyReLU(0.2)),
                                         keras.layers.Dropout(0.4),
                                         keras.layers.Flatten(),
                                         keras.layers.Dense(1, activation="sigmoid")
                                        ], name="Discriminator")

gan = keras.models.Sequential([generator, discriminator], name="GAN")

In [41]:
gan.summary()

Model: "GAN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Generator (Sequential)       (None, 28, 28, 1)         840705    
_________________________________________________________________
Discriminator (Sequential)   (None, 1)                 212865    
Total params: 1,053,570
Trainable params: 1,053,186
Non-trainable params: 384
_________________________________________________________________
